In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
match_results_df = pd.read_csv('../input/dream11-ipl2020-live/IPLMatchResults2020.csv')

In [ ]:
match_results_df.loc[22,'TEAMS'] = 'RR vs DC'

In [ ]:
match_results_df.loc[19,'TEAMS']= 'MI vs RR'

In [ ]:
match_results_df.head()

In [ ]:
match_results_df.isna().sum()

### The Toss

In [ ]:
plt.figure(figsize=(8,5),)
match_results_df['TOSS'].value_counts().plot(kind='bar')
plt.show()

The luckiest team with the toss is SRH. SRH won 11 out of 16 toss.

The unluckiest team with the toss is KXIP. KXIP only won 4 out of 14 tosses.

### Does winning the toss means winning the match?

In [ ]:
won = 0
lost = 0
for i in range(60):
    if match_results_df.iloc[i,-3] == match_results_df.iloc[i,2]:
        won+=1
    else:
        lost+=1

In [ ]:
print(f"Out of 60 macthes, in {won} matches the team that won the toss won the match.\n")
print(f"That is in {round((won/60)*100,2)} % of the matches, team that won the toss won the match.")

In [ ]:
toss_vs_result = pd.DataFrame({'Won_match':[25,35],'Lost_match':[35,25]},index=['Won_toss','Lost_toss'])
toss_vs_result

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
chi2_contingency(toss_vs_result)

The pvalue = 0.10, which is greater than 0.05. Hence we accept null hyposthesis. There is no significant relation between winning the toss and winning the match.

In [ ]:
team_wise_toss_vs_result = pd.crosstab(match_results_df['TOSS'],match_results_df['RESULT (won_by)'])
team_wise_toss_vs_result

In [ ]:
for team in team_wise_toss_vs_result.index:
    toss_won = team_wise_toss_vs_result.loc[team,:].sum()
    match_won = team_wise_toss_vs_result.loc[team,team]
    print(f"In {round((match_won/toss_won)*100,2)} % of the matches of {team},{team} won the TOSS and won the MATCH\n")

### Chase or Defend

In [ ]:
chase = 0
defend = 0
for i in range(60):
    if match_results_df.iloc[i,-3] == match_results_df.iloc[i,2]:
        if match_results_df.iloc[i,3] == 'BOWL':
            chase+=1
        else:
            defend+=1
    else:
        if match_results_df.iloc[i,3] == 'BOWL':
            defend+=1
        else:
            chase+=1

In [ ]:
print(f"{chase} out of 60 times team chasing won the match and \n{defend} out of 60 times team defending won the match.")

That is, chasing and defending had exactly equal probablity in winning a match in IPL 2020.

In [ ]:
for team in match_results_df['RESULT (won_by)'].unique():
    bowl_first = 0
    bat_first = 0
    temp_df = match_results_df[match_results_df['RESULT (won_by)'] == team]
    for i in range(temp_df.shape[0]):
        if temp_df.iloc[i,2] == team:
            if temp_df.iloc[i,3] == 'BOWL':
                bowl_first+=1
            else:
                bat_first+=1
                
        else:
            if temp_df.iloc[i,3] == 'BOWL':
                bat_first+=1
            else:
                bowl_first+=1
                
    print(f"Out of {temp_df.shape[0]} wins by {team}:")
    print(f"{bowl_first} matche(s) were won by bowling first and\n{bat_first} matche(s) were won by batting first.\n\n")

### Stadium

In [ ]:
stadiums = match_results_df['GROUND'].unique()
print(f"{len(stadiums)} used in IPL 2020 were: ")
i = 0
for stadium in stadiums:
    i+=1
    print(f"{i}. {stadium}")

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Number of matches played in each stadium")
sns.countplot(match_results_df['GROUND'])
plt.xlabel("Stadium")
plt.ylabel("No. of matches")
plt.show()

In [ ]:
for stadium in stadiums:
    temp_stadium_df = match_results_df[match_results_df['GROUND'] == stadium]
    chase = 0
    defend = 0
    for i in range(temp_stadium_df.shape[0]):
        if temp_stadium_df.iloc[i,-3] == temp_stadium_df.iloc[i,2]:
            if match_results_df.iloc[i,3] == 'BOWL':
                chase+=1
            else:
                defend+=1
        else:
            if match_results_df.iloc[i,3] == 'BOWL':
                defend+=1
            else:
                chase+=1
            
    print("\nStadium: ",stadium)
    print("---------------------")
    print(f"Total number of matches played in {stadium} stadium: {temp_stadium_df.shape[0]}")
    print(f"Number of wins by chasing a total: {chase}")
    print(f"Number of wins by defending a total: {defend}")
    print(f"Win percentage of chasing a total: {round((chase/temp_stadium_df.shape[0])*100,2)} %")
    print(f"Win percentage of defending a total: {round((defend/temp_stadium_df.shape[0])*100,2)} %")
    print("***********************")

In IPL 2020, teams performed better by defending a total in Sheikh Zayed stadium and teams performed better by chasing a total in Sharjah stadium.

In Dubai stadium, teams performed equally well by chasing and defending a total.

#### Checking statisticaly whether batting first or bowling first had a significant effect on the result in each stadium

Sheikh Zayed Stadium.

In [ ]:
#Sheikh Zayed Stadium.
#Total number of matches played in Sheikh Zayed stadium: 22
#Number of wins by chasing a total: 6
#Number of wins by defending a total: 16
Sheikh_Zayed = pd.DataFrame({"Won":[6,16],"Lost":[16,6]},index=['Chasing','Defending'])
Sheikh_Zayed

Null hypothesis: Chasing or defending did NOT have a significant effect on result of the match in Sheikh Zayed Stadium.

Alternate hypothesis: Chasing or defending did have a significant effect on result of the match in Sheikh Zayed Stadium.

In [ ]:
chi2_contingency(Sheikh_Zayed)

As pvalue = 0.006 is less than 0.05, we reject null hyothesis.

Hence, We can say that there is statistical proof that team defending a total have a better chance of winning in Sheikh Zayed Stadium.

Sharjah Stadium

In [ ]:
#Sharjah Stadium
#Total number of matches played in Sharjah stadium: 12
#Number of wins by chasing a total: 8
#Number of wins by defending a total: 4
Sharjah = pd.DataFrame({"Won":[8,4],"Lost":[4,8]},index=['Chasing','Defending'])
Sharjah

Null hypothesis: Chasing or defending did NOT have a significant effect on result of the match in Sharjah Stadium.

Alternate hypothesis: Chasing or defending did have a significant effect on result of the match in Sharjah Stadium.

In [ ]:
chi2_contingency(Sharjah)

As the pvalue = 0.22 is greater than 0.05, we accept null hypothesis.

Hence, we can say that in Sharjah stadium, we do not have any statistical proof to say that chasing a total has a better chance of winning.

##### First innings and second innings in each stadium

In [ ]:
for stadium in stadiums:
    fi_runs = 0
    fi_wick = 0
    si_runs = 0
    si_wick = 0
    temp_stadium_df = match_results_df[match_results_df['GROUND'] == stadium]
    fi_runs = temp_stadium_df.loc[:,'First_half_score'].apply(lambda x:int(x.split('/')[0]))
    fi_wick = temp_stadium_df.loc[:,'First_half_score'].apply(lambda x:int(x.split('/')[1]))
    si_runs = temp_stadium_df.loc[:,'Second_half_score'].apply(lambda x:int(x.split('/')[0]))
    si_wick = temp_stadium_df.loc[:,'Second_half_score'].apply(lambda x:int(x.split('/')[1]))
    print("\nStadium: ",stadium)
    print("---------------------")
    print(f"Total number of matches played in {stadium} stadium: {temp_stadium_df.shape[0]}")
    print(f"Total Runs scored in {stadium} stadium: {np.sum(fi_runs) + np.sum(si_runs)}")
    print(f"Total Wickets taken in {stadium} stadium: {np.sum(fi_wick)+np.sum(si_wick)}")
    print(f"Average first innings score: {round(np.mean(fi_runs))}")
    print(f"Average wickets taken in first innings: {round(np.mean(fi_wick))}")
    print(f"Average second innings score: {round(np.mean(si_runs))}")
    print(f"Average wickets taken in first second: {round(np.mean(si_wick))}")
    print("***********************")


In [ ]:
match_detailed = match_results_df.copy()

In [ ]:
match_detailed['First_Inns_Runs'] = match_detailed['First_half_score'].apply(lambda x:int(x.split('/')[0]))
match_detailed['First_Inns_Wkts'] = match_detailed['First_half_score'].apply(lambda x:int(x.split('/')[1]))
match_detailed['Second_Inns_Runs'] = match_detailed['Second_half_score'].apply(lambda x:int(x.split('/')[0]))
match_detailed['Second_Inns_Wkts'] = match_detailed['Second_half_score'].apply(lambda x:int(x.split('/')[1]))

In [ ]:
team_batting_first = []
team_batting_second = []
for i in range(match_detailed.shape[0]):
    if match_detailed.loc[i,'CHOOSE_TO'] == 'BOWL':
        team1 = set()
        team1.add(match_detailed.loc[i,'TOSS'])
        teams_playing = set(match_detailed.loc[i,'TEAMS'].split(' vs '))
        team2 = teams_playing - team1
        team_batting_first.append(list(team2)[0])
        team_batting_second.append(list(team1)[0])
    else:
        team1 = set()
        team1.add(match_detailed.loc[i,'TOSS'])
        teams_playing = set(match_detailed.loc[i,'TEAMS'].split(' vs '))
        team2 = teams_playing - team1
        team_batting_first.append(list(team1)[0])
        team_batting_second.append(list(team2)[0])
        

In [ ]:
match_detailed['Team_Batting_First'] = team_batting_first
match_detailed['Team_Batting_Second'] = team_batting_second

In [ ]:
match_detailed = match_detailed[['Team_Batting_First','Team_Batting_Second','TOSS', 'CHOOSE_TO', 'GROUND','First_Inns_Runs', 'First_Inns_Wkts', 'Second_Inns_Runs',
       'Second_Inns_Wkts','RESULT (won_by)', 'Super_over', 'MOM']]

In [ ]:
match_detailed.head()

In [ ]:
match_detailed.groupby('GROUND')[['First_Inns_Runs', 'Second_Inns_Runs']].mean().plot(kind='bar',figsize=(8,7))
plt.show()

In [ ]:
match_detailed.groupby('GROUND')[['First_Inns_Wkts','Second_Inns_Wkts']].mean().plot(kind='bar',figsize=(8,7))
plt.show()

### Teams

###### Statistics of each team batting first

In [ ]:
bat_first_df = match_detailed.groupby('Team_Batting_First').agg({'First_Inns_Runs':[np.mean,np.sum],'First_Inns_Wkts':[np.mean,np.sum]})

In [ ]:
bat_first_df.columns = ['Average runs scored','Total runs scored',
            'Average wicket fall','Total Wicket Fall']

In [ ]:
temp_df = pd.DataFrame(match_detailed.groupby('Team_Batting_First')['Team_Batting_First'].count())
temp_df.columns = ['No. of Matches']

In [ ]:
bat_first_df = pd.concat([temp_df,bat_first_df],axis=1)

In [ ]:
bat_first_df.sort_values('Average runs scored',ascending=False)

In [ ]:
colors = ['yellow','deepskyblue','indigo','indianred','blue','red','violet','orangered']

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Average First Innings Score of each Team")
sns.barplot(y=bat_first_df.sort_index()['Average runs scored'],x=bat_first_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

Mumbai Indians has the highest first innings average.

###### Statistics of each team batting second

In [ ]:
bat_second_df = match_detailed.groupby('Team_Batting_Second').agg({'Second_Inns_Runs':[np.mean,np.sum],'Second_Inns_Wkts':[np.mean,np.sum]})

In [ ]:
bat_second_df.columns = ['Average runs scored','Total runs scored',
            'Average wicket fall','Total Wicket Fall']

In [ ]:
temp_df = pd.DataFrame(match_detailed.groupby('Team_Batting_Second')['Team_Batting_Second'].count())
temp_df.columns = ['No. of Matches']

In [ ]:
bat_second_df = pd.concat([temp_df,bat_second_df],axis=1)

In [ ]:
bat_second_df.sort_values('Average runs scored',ascending=False)

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Average Second Innings Score of each Team")
sns.barplot(y=bat_second_df.sort_index()['Average runs scored'],x=bat_second_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

Chennai Super Kings has the highest second innings score average.

###### Statistics of each team fielding first

In [ ]:
field_first_df = match_detailed.groupby('Team_Batting_Second').agg({'First_Inns_Runs':[np.mean,np.sum],'First_Inns_Wkts':[np.mean,np.sum]})

In [ ]:
field_first_df.columns = ['Average runs conceded','Total runs conceded',
            'Average wickets taken','Total wickets taken']

In [ ]:
temp_df = pd.DataFrame(match_detailed.groupby('Team_Batting_Second')['Team_Batting_Second'].count())
temp_df.columns = ['No. of Matches']

In [ ]:
field_first_df = pd.concat([temp_df,field_first_df],axis=1)

In [ ]:
field_first_df.index.name='Team Fielding First'

In [ ]:
field_first_df.sort_values('Average runs conceded')

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Average Runs Conceded in First Innings")
sns.barplot(y=field_first_df.sort_index()['Average runs conceded'],x=field_first_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

Mumbai Indians has the lowest first innings runs conceded.

###### Statistics of each team fielding second

In [ ]:
field_second_df = match_detailed.groupby('Team_Batting_First').agg({'Second_Inns_Runs':[np.mean,np.sum],'Second_Inns_Wkts':[np.mean,np.sum]})

In [ ]:
field_second_df.columns = ['Average runs conceded','Total runs conceded',
            'Average wickets taken','Total wickets taken']

In [ ]:
temp_df = pd.DataFrame(match_detailed.groupby('Team_Batting_First')['Team_Batting_First'].count())
temp_df.columns = ['No. of Matches']

In [ ]:
field_second_df = pd.concat([temp_df,field_second_df],axis=1)

In [ ]:
field_second_df.index.name = 'Team Fielding Second'

In [ ]:
field_second_df.sort_values('Average runs conceded')

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Average Runs Conceded in Second Innings")
sns.barplot(y=field_second_df.sort_index()['Average runs conceded'],x=field_second_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

CSK has the lowest first innings runs conceded.

###### Difference between runs scored by a team in the first innings and the runs conceded by the team in the second innings

In [ ]:
diff_scored_conceded = pd.DataFrame(bat_first_df.sort_index()['Average runs scored'] - field_second_df.sort_index()['Average runs conceded'])

In [ ]:
diff_scored_conceded.index.name = 'Team'
diff_scored_conceded.columns = ['Runs_scored - Runs_conceded']
diff_scored_conceded.sort_values('Runs_scored - Runs_conceded',ascending=False)

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Difference between Runs Scored in 1st innings and Runs Conceded in 2nd innings")
sns.barplot(y=diff_scored_conceded.sort_index()['Runs_scored - Runs_conceded'],x=diff_scored_conceded.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

When SRH was batting first, the team chasing the target scored around 29 runs lesser than the target on an average.

###### Difference between runs conceded by a team in the first innings and the runs scored by the team in the second innings

In [ ]:
diff_conceded_scored = pd.DataFrame(bat_second_df.sort_index()['Average runs scored']-field_first_df.sort_index()['Average runs conceded'])

In [ ]:
diff_conceded_scored.index.name = 'Team'
diff_conceded_scored.columns = ['Runs_conceded-Runs_scored']
diff_conceded_scored.sort_values('Runs_conceded-Runs_scored',ascending=False)

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Difference between Runs Conceded in 1st innings and Runs Scored in 2nd innings")
sns.barplot(y=diff_conceded_scored.sort_index()['Runs_conceded-Runs_scored'],x=diff_conceded_scored.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

On an average, Mumbai Indians is the only team that has managed to score atleast one run more than the run they conceded in the first innings.

In [ ]:
tot_runs_scored = bat_first_df.sort_index()['Total runs scored'] + bat_second_df.sort_index()['Total runs scored']
tot_wkt_fall = bat_first_df.sort_index()['Total Wicket Fall'] + bat_second_df.sort_index()['Total Wicket Fall']
tot_runs_conceded = field_first_df.sort_index()['Total runs conceded'] + field_second_df.sort_index()['Total runs conceded']
tot_wkts_taken = field_first_df.sort_index()['Total wickets taken'] + field_second_df.sort_index()['Total wickets taken']

In [ ]:
team_summary_df = pd.DataFrame({"Total Runs Scored":tot_runs_scored,'Total Wicket Fall':tot_wkt_fall,
             "Total Runs Conceded":tot_runs_conceded,"Total Wickets Taken":tot_wkts_taken})
team_summary_df.index.name= 'Team'

In [ ]:
team_summary_df

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Total Runs Scored By Each Team")
sns.barplot(y=team_summary_df.sort_index()['Total Runs Scored'],x=team_summary_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Total Runs Conceded By Each Team")
sns.barplot(y=team_summary_df.sort_index()['Total Runs Conceded'],x=team_summary_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Total Wicket Fall Of Each Team")
sns.barplot(y=team_summary_df.sort_index()['Total Wicket Fall'],x=team_summary_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title("Total Wickets Taken by By Each Team")
sns.barplot(y=team_summary_df.sort_index()['Total Wickets Taken'],x=team_summary_df.sort_index().index,palette=colors)
plt.xlabel('Team')
plt.show()